In [1]:
import gurobipy as gp
from gurobipy import GRB

import pandas as pd
import numpy as np

In [2]:
# Input the data
Aij = pd.read_excel('data.xlsx', sheet_name='Aij')
Aij = Aij.iloc[0:5,1:6]
matrix_Aij = np.array(Aij)

Demand = pd.read_excel('data.xlsx', sheet_name='Demand')
Demand = Demand.iloc[0:5,1:2]
matrix_demand = np.array(Demand)

Capacity = pd.read_excel('data.xlsx', sheet_name='Capacity')
Capacity = Capacity.iloc[0:5,1:2]
matrix_capacity = np.array(Capacity)

In [3]:
# Model
m = gp.Model("Ramp_Metering")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-27


In [4]:
# Create decision variables
x1 = m.addVar(name="x1")
x2 = m.addVar(name="x2")
x3 = m.addVar(name="x3")
x4 = m.addVar(name="x4")

y1 = m.addVar(vtype=GRB.BINARY, name="y1")
y2 = m.addVar(vtype=GRB.BINARY, name="y2")
y3 = m.addVar(vtype=GRB.BINARY, name="y3")
y4 = m.addVar(vtype=GRB.BINARY, name="y4")

k1 = m.addVar(vtype=GRB.BINARY, name="k1")
k2 = m.addVar(vtype=GRB.BINARY, name="k2")
k3 = m.addVar(vtype=GRB.BINARY, name="k3")
k4 = m.addVar(vtype=GRB.BINARY, name="k4")

var = [x1, x2, x3, x4, y1, y2, y3, y4, k1, k2, k3, k4]


In [5]:
# Set the objective function
m.setObjective(matrix_demand[0,0]+1.0*x1 + 1.0*x2 + 
               1.0*x3 + 1.0*x4, GRB.MAXIMIZE)

In [6]:
# Add constraints
a = 0
b = 0
M = 1000000000000000000

#1
for j in range (1,5):
    for i in range (1,j+1): 
        a = matrix_Aij[i,j] * var[i-1]
        b = b + a
    m.addConstr(matrix_demand[0,0] * matrix_Aij[0,j] + 
                b <= matrix_capacity[j-1,0])
    b = 0
    
#2   
for j in range (1,5):
    m.addConstr(var[j-1] <= matrix_demand[j,0])
    
#3   
for j in range (1,5):
    m.addConstr(var[j-1] >= matrix_demand[j,0] * (1-var[j+3]))
    
#4
for j in range (1,5):
    m.addConstr(var[j-1] >= 180 * var[j+3])
    
#5
for j in range (1,5):
    m.addConstr(var[j-1] <= matrix_demand[j,0] * (1-var[j+3]) + 
               900 * var[j+3])
    
#6
for j in range (1,5):
    m.addConstr(var[j+3] <= M * var[j+7])
    
#7
for j in range (1,5):
    m.addConstr(var[j-1] - matrix_demand[j,0] <= M * (1-var[j+7]))
    

In [7]:
# Optimize the model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 28 rows, 12 columns and 54 nonzeros
Model fingerprint: 0x2998f0db
Variable types: 4 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+18]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+02, 1e+18]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 28 rows and 12 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 6573.33 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.573333333333e+03, best bound 6.573333333333e+03, gap 0.0000%


In [8]:
# Results
for v in m.getVars():
    print('%s %g' % (v.VarName, v.X))

x1 333.333
x2 180
x3 460
x4 1000
y1 1
y2 1
y3 0
y4 0
k1 1
k2 1
k3 0
k4 0
